# 根据 txt 以及 json一一对应的数据，生成

In [ ]:
import os
import json

# 定义文件目录
directory = os.path.join(os.getcwd(), "prompt_txt_json")# lasson

# 获取目录下所有 .txt 和 .json 文件
txt_files = [f for f in os.listdir(directory) if f.endswith('.txt')]
json_files = [f for f in os.listdir(directory) if f.endswith('.json')]

cnt = 0
result_list = []
# 遍历所有 .txt 文件
for txt_file in txt_files:
    # 找到对应的 .json 文件
    json_file = txt_file.replace('.txt', '_result.json')
    
    if json_file in json_files:
        # 读取 .txt 文件内容
        with open(os.path.join(directory, txt_file), 'r') as file:
            instruction = file.read().strip()

        # 读取 .json 文件内容
        with open(os.path.join(directory, json_file), 'r') as file:
            json_content = json.load(file)
            output = json.dumps(json_content, ensure_ascii=False)
        cnt += 1
        # 提取文件名前缀作为name
        name = txt_file.split('_')[0]
        if "other" in name:
            name = "other_disease"
        elif "pseudo" in name:
            name = "pseudo_papilledema"
        elif "Synonms" in name:
            name = "real_papilledema"
        name = "generate_" + name + "_referral_letter"

        # 创建目标格式的字典
        result = {
            "id": f"seed_task_{cnt}",
            "name": name,
            "instruction": instruction,
            "instances": [{"input": "", "output": output}],
            "is_classification": False
        }
        result_list.append(result)

with open('seed_tasks.jsonl', 'w', encoding='utf-8') as output_file:
    for result in result_list:
        output_file.write(json.dumps(result, ensure_ascii=False) + '\n')

In [9]:
print(result_list)

[{'id': 'seed_task_1', 'name': 'other', 'instruction': 'Role: You are an experienced doctor with knowledge of electronic medical records related to many diseases.\n\nInstruction: Please generate a referral letter for a non-ophthalmic disease. The letter should be within 150 words and include a clear referral reason along with some basic patient information. The referral reason should be in a separate paragraph dedicated entirely to it. Do not use a table for patient information. If a table is used, it should be placed in the middle of the letter. Use different beginnings and endings for different letters. The beginning and ending of the letter should be brief, with minimal email and address details, and include simulated information.\n\nOutput your result in the format: {"is_referral_letter": boolean, "is_Papilledema": boolean, "whole_letter": "", "referral_content": ""}\n\nRule For is_Papilledema: If the referral letter contains any words from the provided Synonyms, then is_Papilledem

# 使用 mak的700多条数据，生成prompt，放入seed_tasks.jsonl文件里

In [2]:
def make_single_instruction_using_mak(txt):
    instruction = f"""Role : You are a experienced doctor who have memory of electronic medical records related to many diseases.

    Instruction : please generate one referral letter according to the medical records separeted by ###. It need to be within 150 words.And it should include the very clear referral reason and few simulated patient basic information. The referral reason should be in a whole separate paragraph, which means this paragraph shoule be all about the referral reason. Also the referral reason should be short and precise.You don't use table to contain patient information. If use table, This table should be in the middle part of the letter. Just use different begining and ending information in different letters.The letter begining as well as ending should be short, which means the email and address can be as short as possible. And the it should have simulated information. 

    ###
    {txt}
    ###

    output your result in format: {{"is_referral_letter": boolean, "is_Papilledema": boolean, "whole_letter": "", "referral_content": "",}}

    Rule For is_Papilledema : false.

    Rule For is_referral_letter : if you are producing a referral letter, then it is true.

    Rule For whole_letter : this is the content you generate.

    Rule For referral_content : this content should be a whole paragraph which tells Patient need referral. If the referral_letter contains this content, you should include it. If the letter doesn't contain related information, then it should be null.
    """
    return instruction


In [3]:
import pandas as pd
import json

# 加载 CSV 文件
file_path = './prompt_txt_json/filtered_data_unique_4.csv'
data = pd.read_csv(file_path)

# 构造基于 'patient' 列的 JSON 数据
cnt = 13 # 因为从13开始
result_list = []

# 遍历每一行并构造 JSON 结构
for index, row in data.iterrows():    
    instruction = make_single_instruction_using_mak(row['patient'].strip())
    output = json.dumps({
            "is_referral_letter": True,
            "is_Papilledema": False, # 只有索引为4， seed_task_16的数据，此处为true。看gpt查出来的，所以手动改了数据，而没写代码
            "whole_letter": "nothing",
            "referral_content": "nothing"
    }, ensure_ascii=False)
    result = {
        "id": f"seed_task_{cnt}",
        "name": "generate_mak_other_disease_referral_letter",
        "instruction": instruction,
        "instances": [{"input": "", "output": output}],
        "is_classification": False
    }
    
    result_list.append(result)
    cnt += 1

In [4]:
data.head()

,patient
0,Patient 1: A 64-year-old man with a left infer...
1,A 15-year-old boy came to the emergency depart...
2,A 13-year-old male patient had a history of pr...
3,"Case 1: An eight-year-old, healthy, African-Am..."
4,A 39-year-old female patient underwent surgery...


In [4]:
print(len(result_list))
print(result_list[0])

723
{'id': 'seed_task_13', 'name': 'generate_mak_other_disease_referral_letter', 'instruction': 'Role : You are a experienced doctor who have memory of electronic medical records related to many diseases.\n\n    Instruction : please generate one referral letter according to the medical records separeted by ###. It need to be within 150 words.And it should include the very clear referral reason and few simulated patient basic information. The referral reason should be in a whole separate paragraph, which means this paragraph shoule be all about the referral reason. Also the referral reason should be short and precise.You don\'t use table to contain patient information. If use table, This table should be in the middle part of the letter. Just use different begining and ending information in different letters.The letter begining as well as ending should be short, which means the email and address can be as short as possible. And the it should have simulated information. \n\n    ###\n    P

In [5]:
# 将 JSON 数据保存到文件
with open('seed_tasks.jsonl', 'a', encoding='utf-8') as output_file:
    for result in result_list:
        output_file.write(json.dumps(result, ensure_ascii=False) + '\n')

# 重新读取 seed_tasks.jsonl 看看每条数据的格式是否正确

In [6]:
import json
data = []
with open('seed_tasks.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.strip()))

In [7]:
print(len(data))

735


## 读取第一条数据，查看形式

In [12]:
sample = data[0]
output = sample["instances"][0]["output"]
print(type(output))
print(output)

<class 'str'>
{"is_referral_letter": true, "is_Papilledema": false, "whole_letter": "Dr. Sarah Thompson\nABC Medical Clinic\n789 Oak St, Suite 204\nTown, State, 34567\nOctober 5, 2024\n\nDr. John Miller\nNeurology Specialists\n123 Pine Rd, Suite 101\nTown, State, 67890\n\nDear Dr. Miller,\n\nI am referring Ms. Jane Doe, a 60-year-old female, for urgent evaluation due to suspected multiple sclerosis. Ms. Doe has been experiencing numbness, tingling, and muscle weakness over the past month.\n\n**Referral Reason:**\nBased on the patient's symptoms and initial examination findings, there is a high suspicion of multiple sclerosis. An MRI showed multiple demyelinating lesions in the brain. Given her family history of neurological disorders, further neurological assessment is necessary.\n\nYour expertise is needed to confirm the diagnosis and develop an appropriate treatment plan to manage her condition.\n\nBest regards,\nDr. Sarah Thompson\nABC Medical Clinic", "referral_content": "Based on 

In [9]:
output_dict = json.loads(output)
print(type(output_dict))

<class 'dict'>


In [10]:
print(output_dict["is_referral_letter"])
print(output_dict["is_Papilledema"])

print(output_dict["referral_content"])

True
False
Based on the patient's symptoms and initial examination findings, there is a high suspicion of multiple sclerosis. An MRI showed multiple demyelinating lesions in the brain. Given her family history of neurological disorders, further neurological assessment is necessary.


## 读取第16条数据，查看形式

In [11]:
sample_last = data[16]
output = sample_last["instances"][0]["output"]
print(type(output))
print(output)

output_dict = json.loads(output)
print(type(output_dict))
print(output_dict["is_referral_letter"])
print(output_dict["is_Papilledema"])

print(output_dict["referral_content"])

<class 'str'>
{"is_referral_letter": true, "is_Papilledema": false, "whole_letter": "nothing", "referral_content": "nothing"}
<class 'dict'>
True
False
nothing


# 读取 ./lasson/regen.json

## 清理每条regen里边的数据，并且加上 seed_tasks里边对应的信息

In [13]:
import json
seed_tasks_data = []
with open('seed_tasks.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        seed_tasks_data.append(json.loads(line.strip()))

In [14]:
import json
with open('./lasson/regen.json', 'r', encoding='utf-8') as f:
    regen_data = json.loads(f.read())

In [15]:
print(len(regen_data))

735


In [17]:
import json
import re
seed_tasks_data = []
with open('seed_tasks.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        seed_tasks_data.append(json.loads(line.strip()))

with open('./lasson/regen.json', 'r', encoding='utf-8') as f:
    regen_data = json.loads(f.read())
        
# 用于存储成功处理的数据
updated_regen_data = []

for i, regen_sample in enumerate(regen_data):
    regen_output = regen_sample["output"]
    
    if re.findall(r"^```json\n|```\n$", regen_output):
        
        # 清理开头和结尾的 ```json 和 ```\n
        regen_output = re.sub(r'^```json\n', '', regen_output, flags=re.MULTILINE)
        regen_output = re.sub(r"\n```$", '', regen_output, flags=re.MULTILINE)
        
        # 清理四个字段后边的 \n
        regen_output = re.sub(r',\n\s+"is_Papilledema"', ', "is_Papilledema"', regen_output)
        regen_output = re.sub(r',\n\s+"whole_letter"', ', "whole_letter"', regen_output)
        regen_output = re.sub(r',\n\s+"referral_content"', ', "referral_content"', regen_output)
        regen_output = re.sub(r'\n}', '}', regen_output)
        

#         print(type(regen_output))
#         print(regen_output)
    try:
        regen_output_json = json.loads(regen_output)
        a = regen_output_json["is_Papilledema"]
        b = regen_output_json["is_referral_letter"]
        c = regen_output_json["whole_letter"]
        d = regen_output_json["referral_content"]
        
         # 添加 instruction 字段
        regen_sample["gpt_generate_data_instruction"] = seed_tasks_data[i]["instruction"]
        
        # 添加 id 字段
        regen_sample["id"] = seed_tasks_data[i]["id"]
        
        # 将更新后的 JSON 对象转换回字符串并存储回 regen_sample 中
        regen_sample["output"] = json.dumps(regen_output_json, ensure_ascii=False, indent=4)
        updated_regen_data.append(regen_sample)

    except json.JSONDecodeError:
        print(i + 1)
#         print(regen_output)
        print()

# 将更新后的 regen_data 写回到一个新的文件
with open('./lasson/updated_regen.json', 'w', encoding='utf-8') as f:
    json.dump(updated_regen_data, f, ensure_ascii=False, indent=4)

1

15

31

34

37

75

77

79

109

187

210

220

247

259

303

343

362

405

415

441

468

476

490

500

516

520

571

579

580

603

614

637

668

677

678

701

722



In [18]:
print(len(updated_regen_data))

698


## 创建 CSV 文件

In [19]:
import pandas as pd
# 创建 CSV 文件
csv_data = []

for item in updated_regen_data:
    output_json = json.loads(item["output"])
    csv_data.append({
        "id": item["id"],
        "name": item["name"],
        "gpt_generate_data_instruction": item["gpt_generate_data_instruction"],
        "is_Papilledema": output_json["is_Papilledema"],
        "whole_letter": output_json["whole_letter"],
        "referral_content": output_json["referral_content"]
    })

# 创建 DataFrame
df = pd.DataFrame(csv_data)

# 保存为 CSV 文件
csv_file_path = './lasson/updated_regen.csv'
df.to_csv(csv_file_path, index=False, encoding='utf-8')

print(f"CSV file has been saved to {csv_file_path}")

CSV file has been saved to ./lasson/updated_regen.csv


In [3]:
df.head(2)

,id,name,gpt_generate_data_instruction,is_Papilledema,whole_letter,referral_content
0,seed_task_1,generate_other_disease_referral_letter,Role: You are an experienced doctor with knowl...,False,Dr. Jane Smith\nSunrise Medical Center\n123 He...,Mr. Doe has recently exhibited symptoms includ...
1,seed_task_2,generate_other_disease_referral_letter,Role: You are an experienced doctor familiar w...,False,Dr. Amy Johnson\nHealthy Living Medical Center...,Mr. Smith has been experiencing persistent joi...


In [22]:
# regen_sample = regen_data[10] # 3有json这个字符， 10 前后有\n
# regen_output = regen_sample["output"]
# print(type(regen_output))
# print(regen_output)

# regen_output_json = json.loads(regen_output)

# print(regen_output_json["is_referral_letter"])
# print(regen_output_json["is_Papilledema"])
# print(regen_output_json["whole_letter"])
# print(regen_output_json["referral_content"])
# print()

# 将数据处理成 llama2可以直接使用的形式

## 添加instruction以及分割数据集

In [20]:
import pandas as pd

df = pd.read_csv("./lasson/updated_regen.csv")

In [21]:
df.head()

,id,name,gpt_generate_data_instruction,is_Papilledema,whole_letter,referral_content
0,seed_task_2,generate_other_disease_referral_letter,Role: You are an experienced doctor familiar w...,False,Dr. John Doe\nGreen Valley Clinic\n1234 Elm St...,Despite adherence to current treatment protoco...
1,seed_task_3,generate_other_disease_referral_letter,Role: You are an experienced doctor with acces...,False,Dr. John Smith\nGeneral Practice\n123 Health B...,Mrs. Thompson has been experiencing frequent e...
2,seed_task_4,generate_other_disease_referral_letter,"Synonyms: [""Papilledema"",""Swollen discs"",""Indi...",False,"Dr. Green,\n\nPlease see the patient below for...",The primary reason for this referral is the pa...
3,seed_task_5,generate_pseudo_papilledema_referral_letter,"Synonyms: [""Papilledema"", ""Swollen discs"", ""In...",False,Dr. Jane Smith\nEyeCare Specialists Clinic\n12...,The referral is due to the presence of anomalo...
4,seed_task_6,generate_pseudo_papilledema_referral_letter,"Synonyms: [""Papilledema"", ""Swollen discs"", ""In...",False,"Dr. Smith,\n\nI hope this letter finds you wel...",The patient presents with optic disc anomalies...


In [22]:
txt = """Synonyms: ["Papilledema","Swollen discs","Indistinct margins","Blurred disc margins","Suspicious discs","Disc swelling","Optic nerve swelling"]

PseudoSynonyms: ["Pseudopapilledema","Drusen","Tilted disc","Anomalous discs"]


Role : You are a experienced doctor who have memory of electronic medical records related to many diseases.

Instruction : please extract the referral reason from the following referral letter  separeted by ###. output your result

output your result directly in format: {"referral_content": "",}

Rule For is_Papilledema : If the referral letter contains one of words in Synonyms, then is_Papilledema = true; If the letter contains words in PseudoSynonyms or doesn't contain words in Synonyms, then is_Papilledema = False.


Rule For referral_content : this content should be a whole paragraph which tells Patient need referral. If the referral_letter contains this content, you should include it. If the letter doesn't contain related information, then it should be null."""

df["instruction"] = txt
# 保存为 CSV 文件
# csv_file_path = './lasson/updated_regen.csv'
# df.to_csv(csv_file_path, index=False, encoding='utf-8')

# print(f"CSV file has been saved to {csv_file_path}")

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
df = df[["instruction", "whole_letter", "referral_content"]]
train_data, test_data = train_test_split(df, test_size=0.2, random_state=2024)

In [25]:
train_data.head()

,instruction,whole_letter,referral_content
258,"Synonyms: [""Papilledema"",""Swollen discs"",""Indi...","Dear Dr. Smith,\n\nI am writing to refer a 10-...",Referral Reason: The findings suggest the pres...
492,"Synonyms: [""Papilledema"",""Swollen discs"",""Indi...","Dear Colleague,\n\nI am writing to refer Mr. J...",I believe Mr. Doe requires an urgent consultat...
52,"Synonyms: [""Papilledema"",""Swollen discs"",""Indi...","Dear Colleague,\n\nI am referring Mr. John Doe...",The reason for this referral is to request you...
297,"Synonyms: [""Papilledema"",""Swollen discs"",""Indi...","Dear Colleague,\n\nI am writing to refer Mr. J...",Given the specificity and urgency of his condi...
531,"Synonyms: [""Papilledema"",""Swollen discs"",""Indi...","Dear Dr. Smith,\n\nI hope this message finds y...",The referral is specifically for the managemen...


In [26]:
df.columns

Index(['instruction', 'whole_letter', 'referral_content'], dtype='object')

## 保存train

In [27]:
import csv
import json
import os

if not os.path.exists("../lasson_LLM/train_test_data_735/"):
    os.makedirs("../lasson_LLM/train_test_data_735/")


# 定义输入 CSV 文件和输出 JSONL 文件的路径
output_jsonl_file = '../lasson_LLM/train_test_data_735/train.jsonl'


# 读取 CSV 文件并写入 JSONL 文件
with open(output_jsonl_file, mode='w', encoding='utf-8') as train_jsonl_file:
    for index, row in train_data.iterrows():
        train_jsonl_file.write(json.dumps(row.to_dict(), ensure_ascii=False) + '\n')

print(f"CSV data has been converted to JSONL format and saved to {output_jsonl_file}.")


CSV data has been converted to JSONL format and saved to ../lasson_LLM/train_test_data_735/train.jsonl.


## 保存 test

In [28]:
# 定义输入 CSV 文件和输出 JSONL 文件的路径
output_jsonl_file = '../lasson_LLM/train_test_data_735/test.jsonl'

if not os.path.exists("../lasson_LLM/train_test_data_735/"):
    os.makedirs("../lasson_LLM/train_test_data_735/")

# 读取 CSV 文件并写入 JSONL 文件
with open(output_jsonl_file, mode='w', encoding='utf-8') as test_jsonl_file:
    for index, row in test_data.iterrows():
        test_jsonl_file.write(json.dumps(row.to_dict(), ensure_ascii=False) + '\n')

print(f"CSV data has been converted to JSONL format and saved to {output_jsonl_file}.")

CSV data has been converted to JSONL format and saved to ../lasson_LLM/train_test_data_735/test.jsonl.
